# ASTR 598 Astrostatistics
## HW2 Part 1
## Hayden Smotherman, Chris Suberlack, Winnie Wang
## To run this Notebook:

Run the file to calculate distances for part A and B.
Still working on parts C and D

In [1]:
# Imports 
%matplotlib inline
from astropy.table import Table
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.coordinates import Galactocentric
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import CartesianDifferential
from astropy import units as u
from astropy.table import hstack
from astropy.table import vstack

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

import os
import numpy as np
import math
from astropy.io import fits
import pandas as pd
from scipy.stats import binned_statistic_2d as bs2d 
from scipy.stats import binned_statistic as bs1d

In [2]:
# change given apparent and absolute magnitude into distance modulus: 
# m - M = 5*log(d) - 5
m = 21
M = 5
ra_deg = 0
dec_deg = 0

### Part A:

The heliocentric distance is simply obtained from changing the distance modulus into distance through the formula:
$(m - M) = 5*log(d) - 5$:

In [3]:
d = 10 ** ((m - M + 5) / 5) #in parsecs
d_helio = (d/1000) * u.kpc

#print(d)
print("Heliocentric distance: "+ str(d_helio) + " for m = 21, Mr = 5")

Heliocentric distance: 15.84893192461114 kpc for m = 21, Mr = 5


### Part B:

To obtain the galactocentric distance to the halo stars that we are observing, we need to transform the heliocentric distance obtained from part A into galactocentric coordinate system, which we can use to change to distance.

In [4]:
# Galactocentric coordinates:
# X = R − D cos(l) cos(b)
# Y = −D sin(l) cos(b)
# Z = D sin(b)
# where D = distance to the sun (heliocentric distance)
# distance = sqrt(X^2 + Y^2 + Z^2)

c = coord.ICRS(ra=[ra_deg] * u.degree, dec=[dec_deg] * u.degree, distance=[d_helio])

# need to change the speed of Sun based on Schronich et al. (2010) ---> astropy already corrected for that
# galcen_v_sun_bond=CartesianDifferential(11.1 * (u.km / u.s), 12.24 * (u.km / u.s), 7.25 * (u.km / u.s))
c2 = c.transform_to(coord.Galactocentric(galcen_distance=8.1*u.kpc)) #gives x,y,z

print(c2)
print(c2.x)

d_galacto = np.sqrt((c2.x)**2 + (c2.y)**2 + (c2.z)**2)
print("Galactocentric distance: "+ str(d_galacto) + " for m=21, Mr=5")

<Galactocentric Coordinate (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    ( 266.4051, -28.936175)>, galcen_distance=8.1 kpc, galcen_v_sun=( 11.1,  232.24,  7.25) km / s, z_sun=27.0 pc, roll=0.0 deg): (x, y, z) in kpc
    [(-9.01548234,  7.83112784, -13.72159591)]>
[-9.01548234] kpc
Galactocentric distance: [ 18.19031829] kpc for m=21, Mr=5


### Part C:

To generate the radial velocity, the distributions were based on Bond et al. (2010) in section 5.1:

In [5]:
# part C; make distribution
# not done yet
from astropy.coordinates import CylindricalRepresentation

def mean_velocity (seed, mean, sig, size):
    np.random.seed(seed) #resets seed
    mu = mean
    sigma = sig
    dist = np.random.normal(mu, sigma, size)
    dist_mean = np.mean(dist)
    
    for i in range(len(dist)):
        sum_squares = (dist[i] ** 2)
    
    dist_rms = (sum_squares) / len(dist)
    
    return dist_mean, dist_rms #returns mean and RMS of value

radial_mean, radial_rms = (mean_velocity(42, 0, 135, 1000)) * (u.km / u.s) #mean and RMS radial velocity
phi_mean, phi_rms = (mean_velocity(43, 0 ,85, 1000)) * u.rad
vertical_mean, vertical_rms = (mean_velocity(44, 0, 85, 1000)) * (u.km / u.s)

cartesian_velocity = CylindricalRepresentation(radial_mean, 
                                               phi_mean, vertical_mean).to_cartesian()
cartesian_vel = cartesian_velocity.norm()
print(cartesian_velocity)
print(cartesian_vel) # magnitude of velocity

# tangential velocity
tan_vel = np.sqrt((cartesian_vel ** 2) - (radial_mean ** 2))
print(tan_vel)

# v_tan = 4.74 * mu * d, where d-> distance
proper_motion = (tan_vel / (4.74 * d_galacto)) #also comment that this is slide 6 from lecture 8, Astr 511 Zeljko

# median -> mean
# rms -> mean * sqrt(2)

# basically:
# 1. obtain v_x, v_y, v_z; change into v_l and v_b
# 2. change ra=0, dec=0 into (l,b)
# coord = SkyCoord(ra=0*u.degree, dec=0*u.degree, frame='icrs')
# coord.galactic ---> gives me (l,b)
# then change 

( 2.49624616,  0.7615477, -0.72100193) km / s
2.7075899882173173 km / s
0.721001925488445 km / s


### Part D:

Median radial velocity:

In [6]:
# median radial velocity is radial_mean
radial_mean

<Quantity 2.6098275360139427 km / s>

RMS radial velocity:

In [7]:
# RMS radial velocity is radial_rms
radial_rms

<Quantity 5.975085231438731 km / s>

In [8]:
#-----other-----#
#helio_distance = (1 * u.AU).to(u.pc)
#galac_distance = (8.1* 1000) * u.pc

c = coord.ICRS(ra=[0] * u.degree, 
                dec=[0] * u.degree,
                distance=[d] * u.kpc)

c2 = c.transform_to(coord.Galactocentric(galcen_distance=8.1*u.kpc)) #gives x,y,z
c3 = c.transform_to(coord.HeliocentricTrueEcliptic) 

#print(d_pc)
#print(helio_distance)
#print(galac_distance)
print(c2) #galacto
print()
print(c3)
#-----other-----#

<Galactocentric Coordinate (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    ( 266.4051, -28.936175)>, galcen_distance=8.1 kpc, galcen_v_sun=( 11.1,  232.24,  7.25) km / s, z_sun=27.0 pc, roll=0.0 deg): (x, y, z) in kpc
    [(-923.62729018,  7831.12784131, -13748.56891313)]>

<HeliocentricTrueEcliptic Coordinate (equinox=J2000.000): (lon, lat, distance) in (deg, deg, kpc)
    [( 359.99613191,  -5.73998824e-06,  15848.93192461)]>
